# Segmenting and Clustering Neighborhoods in Toronto

##  Step 1 : Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

## Import Libraries

In [1]:
#high-level mathematical functions
import numpy as np 
#data manipulation and analysis
import pandas as pd

In [2]:
# To build the code to scrape the following Wikipedia page,
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
#read the data from url
toronto_df, = pd.read_html(wiki_url, match="Postal code", skiprows=1)
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood (given)
toronto_df.columns = ["PostalCode", "Borough", "Neighborhood"]
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights


In [4]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.(given)
toronto_df = toronto_df[toronto_df["Borough"] != "Not assigned"]
toronto_df.head()

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights
5,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
borough_fn = lambda b: b.iloc[0]
neighborhood_fn = lambda b: ", ".join(b)
all_fns = {"Borough": borough_fn, "Neighborhood": neighborhood_fn}
temp_toronto_df = toronto_df.groupby(by="PostalCode").aggregate(all_fns)
temp_toronto_df.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [6]:
toronto_df = temp_toronto_df.reset_index()[toronto_df.columns]
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for (j, row) in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        borough_each = row["Borough"]
        print("Replace \"Not assigned\" => %s in row %i" % (borough_each, j))
        row["Neighborhood"] = borough_each

In [8]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
for (j, row) in toronto_df.iterrows():
     row['Neighborhood']=row['Neighborhood'].replace('/',',')

In [9]:
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [10]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
toronto_df.shape

(103, 3)

## Step 2 : In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

In [11]:
#read csv file (given)
geo_loc_df = pd.read_csv("https://cocl.us/Geospatial_data")
geo_loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#Rename the columns as given format
geo_loc_df.columns = ["PostalCode", "Latitude", "Longitude"]

#Merge both dataframes
toronto_geo_df = pd.merge(toronto_df, geo_loc_df, on='PostalCode')
toronto_geo_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
#Checking data
toronto_geo_df[toronto_geo_df["PostalCode"] == "M5G"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [14]:
toronto_geo_df[toronto_geo_df["PostalCode"] == "M4B"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
35,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937


In [15]:
toronto_geo_df[toronto_geo_df["PostalCode"] == "M9V"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


looks like the downloaded data match with the given (coursera assignment / Geocoder package or the csv file to create the following dataframe) data

## Step:3  Explore and cluster the neighborhoods in Toronto

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [17]:
#Quickly examine the resulting dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_geo_df['Borough'].unique()),
        toronto_geo_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto City

In [18]:
address = 'Toronto, T.O.'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Create a map of toronto with neighborhoods superimposed on top

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_df['Latitude'], toronto_geo_df['Longitude'], toronto_geo_df['Borough'], toronto_geo_df['Neighborhood']):
    label = '{}, {}'.format(toronto_geo_df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's explore the first neighborhood (East York ) in our dataframe.

In [20]:
East_York_df = toronto_geo_df[toronto_geo_df['Borough'] == 'East York'].reset_index(drop=True)
East_York_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,East Toronto,43.685347,-79.338106


In [21]:
address_ey = 'East York, T.O.'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_ey)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 39.9737093, -76.6863554.


In [22]:
# create map of East York using latitude and longitude values
map_eastyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(East_York_df['Latitude'], East_York_df['Longitude'], East_York_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eastyork)  
    
map_eastyork

In [52]:
# @hidden_cell


#Define Foursquare Credentials and Version

CLIENT_ID = '5ZPBJIRPKMQRRXRKE50SHZZNHSUTNK0XBITDQ0QDGVISVATC' # your Foursquare ID
CLIENT_SECRET = 'MEMRRZ41JMIFQMDGECTEGBL0JIZUISP5N3TSDO305Z52PQY5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


In [24]:
East_York_df.loc[0, 'Neighborhood']

'Parkview Hill , Woodbine Gardens'

In [25]:
neighborhood_latitude = East_York_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = East_York_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = East_York_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkview Hill , Woodbine Gardens are 43.7063972, -79.309937.


In [26]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=5ZPBJIRPKMQRRXRKE50SHZZNHSUTNK0XBITDQ0QDGVISVATC&client_secret=MEMRRZ41JMIFQMDGECTEGBL0JIZUISP5N3TSDO305Z52PQY5&v=20180605&ll=43.7063972,-79.309937&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92163540a7ea001b04f361'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': "O'Connor - Parkview",
  'headerFullLocation': "O'Connor - Parkview, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 43.7108972045, 'lng': -79.30372360313615},
   'sw': {'lat': 43.701897195499996, 'lng': -79.31615039686386}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5a3842f964a52023b528e3',
       'name': 'Jawny Bakers',
       'location': {'address': "804 O'Connor Dr",
        'crossStreet': 'St Clair E',
        'lat': 43.705782646822,
        'lng': -79.31291304477831,
        'labeledLatLngs':

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
#Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jawny Bakers,Gastropub,43.705783,-79.312913
1,East York Gymnastics,Gym / Fitness Center,43.710654,-79.309279
2,Shoppers Drug Mart,Pharmacy,43.705933,-79.312825
3,TD Canada Trust,Bank,43.705740,-79.312270
4,Pizza Pizza,Pizza Place,43.705159,-79.313130


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


### Explore Neighborhoods in East York

In [31]:
# Let's create a function to repeat the same process to all the neighborhoods in east york

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called eastyork_venues.

In [32]:
eastyork_venues = getNearbyVenues(names=East_York_df['Neighborhood'],
                                   latitudes=East_York_df['Latitude'],
                                   longitudes=East_York_df['Longitude']
                                  )

Parkview Hill , Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto


Let's check the size of the resulting dataframe

In [33]:
print(eastyork_venues.shape)
eastyork_venues.head()

(75, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
3,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
4,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place



Let's check how many venues were returned for each neighborhood

In [34]:
eastyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
East Toronto,3,3,3,3,3,3
Leaside,33,33,33,33,33,33
"Parkview Hill , Woodbine Gardens",11,11,11,11,11,11
Thorncliffe Park,18,18,18,18,18,18
Woodbine Heights,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(eastyork_venues['Venue Category'].unique())))

There are 47 uniques categories.


### Analyze Each Neighborhood

In [36]:
# one hot encoding
eastyork_onehot = pd.get_dummies(eastyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastyork_onehot['Neighborhood'] = eastyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastyork_onehot.columns[-1]] + list(eastyork_onehot.columns[:-1])
eastyork_onehot = eastyork_onehot[fixed_columns]

eastyork_onehot.head()

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Juice Bar,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,"Parkview Hill , Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Parkview Hill , Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Parkview Hill , Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,"Parkview Hill , Woodbine Gardens",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Parkview Hill , Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [37]:
eastyork_onehot.shape

(75, 48)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
eastyork_grouped = eastyork_onehot.groupby('Neighborhood').mean().reset_index()
eastyork_grouped

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Juice Bar,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Leaside,0.000000,0.030303,0.060606,0.030303,0.030303,0.030303,0.030303,0.060606,0.000000,0.0,0.090909,0.000000,0.0,0.0,0.0,0.030303,0.030303,0.0,0.030303,0.000000,0.030303,0.060606,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.030303,0.030303,0.030303,0.000000,0.030303,0.000000,0.000000,0.030303,0.030303,0.030303,0.0,0.090909,0.030303,0.030303,0.030303,0.0,0.000000,0.000000
2,"Parkview Hill , Woodbine Gardens",0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.181818,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Thorncliffe Park,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.0,0.055556,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.055556,0.055556,0.055556,0.111111,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.0,0.055556,0.055556
4,Woodbine Heights,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.1,0.1,0.1,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000


Let's confirm the new size

In [39]:
eastyork_grouped.shape

(5, 48)

Let's print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in eastyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = eastyork_grouped[eastyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Toronto----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2   Convenience Store  0.33
3  Athletics & Sports  0.00
4          Restaurant  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2                    Bank  0.06
3            Burger Joint  0.06
4  Furniture / Home Store  0.06


----Parkview Hill , Woodbine Gardens----
                venue  freq
0         Pizza Place  0.18
1  Athletics & Sports  0.09
2            Bus Line  0.09
3            Pharmacy  0.09
4           Pet Store  0.09


----Thorncliffe Park----
                  venue  freq
0     Indian Restaurant  0.11
1        Sandwich Place  0.06
2  Gym / Fitness Center  0.06
3                   Gym  0.06
4         Grocery Store  0.06


----Woodbine Heights----
         venue  freq
0  Curling Ice   0.1
1   Beer Store   0.1
2  Video Store   0.1
3         Park   0.1
4     Bus Stop   0.1




Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eastyork_grouped['Neighborhood']

for ind in np.arange(eastyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Convenience Store,Park,Coffee Shop,Cosmetics Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Diner,Dessert Shop,Department Store
1,Leaside,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Liquor Store,Mexican Restaurant,Dessert Shop,Electronics Store,Fish & Chips Shop
2,"Parkview Hill , Woodbine Gardens",Pizza Place,Gastropub,Bank,Bus Line,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports
3,Thorncliffe Park,Indian Restaurant,Gym / Fitness Center,Gym,Coffee Shop,Grocery Store,Warehouse Store,Park,Burger Joint,Pharmacy,Pizza Place
4,Woodbine Heights,Pharmacy,Video Store,Diner,Beer Store,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Bus Stop,Park


### Cluster Neighborhoods

In [43]:
# set number of clusters
kclusters = 5

eastyork_grouped_clustering = eastyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eastyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 4, 1, 0], dtype=int32)

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

eastyork_merged = East_York_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastyork_merged = eastyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

eastyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Gastropub,Bank,Bus Line,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Pharmacy,Video Store,Diner,Beer Store,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Bus Stop,Park
2,M4G,East York,Leaside,43.709060,-79.363452,3,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Liquor Store,Mexican Restaurant,Dessert Shop,Electronics Store,Fish & Chips Shop
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Indian Restaurant,Gym / Fitness Center,Gym,Coffee Shop,Grocery Store,Warehouse Store,Park,Burger Joint,Pharmacy,Pizza Place
4,M4J,East York,East Toronto,43.685347,-79.338106,2,Convenience Store,Park,Coffee Shop,Cosmetics Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Diner,Dessert Shop,Department Store


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastyork_merged['Latitude'], eastyork_merged['Longitude'], eastyork_merged['Neighborhood'], eastyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster : 1

In [46]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 0, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East York,0,Pharmacy,Video Store,Diner,Beer Store,Skating Rink,Dance Studio,Curling Ice,Cosmetics Shop,Bus Stop,Park


Cluster 2

In [47]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 1, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East York,1,Indian Restaurant,Gym / Fitness Center,Gym,Coffee Shop,Grocery Store,Warehouse Store,Park,Burger Joint,Pharmacy,Pizza Place


Cluster 3

In [48]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 2, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East York,2,Convenience Store,Park,Coffee Shop,Cosmetics Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Diner,Dessert Shop,Department Store


Cluster 4

In [49]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 3, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East York,3,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Liquor Store,Mexican Restaurant,Dessert Shop,Electronics Store,Fish & Chips Shop


Cluster 5

In [50]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 4, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East York,4,Pizza Place,Gastropub,Bank,Bus Line,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports
